# Data Preprocessing for multi-class Classification 
# in the domain of TCGA-STAD (The Cancer Genome Atlas Stomach Adenocarcinoma)

## 0. Making Pandas dataframe including only X, Y values needed by model 

In [1]:
import pandas as pd
import numpy as np
import h5py

### 0-1. Downloading and exploring 2 tsv files from Xenabrowser

In [3]:
df1 = pd.read_csv('TCGA-STAD.htseq_fpkm-uq_transposed.tsv', sep='\t')
df2 = pd.read_csv('TCGA-STAD.GDC_phenotype.tsv', sep='\t')

In [4]:
# 407명 기증자의 모든 유전자 위치에서의 mRNA 발현량  
df1

,Ensembl_ID,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,ENSG00000198242.12,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
0,TCGA-D7-5577-01A,0.000000,0.000000,17.440424,0.0,17.929620,18.847523,0.000000,10.845012,23.646398,...,0.0,11.483109,0.000000,13.398034,0.000000,19.959486,10.999693,0.00000,14.477365,0.000000
1,TCGA-D7-6818-01A,0.000000,7.328476,16.894902,0.0,16.047477,18.393746,0.000000,12.540158,22.206059,...,0.0,17.356251,0.000000,13.663386,0.000000,20.320092,13.067673,0.00000,14.652790,0.000000
2,TCGA-BR-7958-01A,0.000000,0.000000,16.724229,0.0,17.667541,18.118569,0.000000,10.439250,22.500445,...,0.0,9.883739,0.000000,13.037143,0.000000,19.314249,11.486559,0.00000,14.601858,0.000000
3,TCGA-D7-8572-01A,10.766176,8.242639,14.805945,0.0,17.222161,18.504515,10.098788,13.005000,22.130398,...,0.0,14.306127,9.840922,13.452516,11.824639,18.728202,12.738185,11.34276,13.842675,10.666970
4,TCGA-VQ-A91Z-01A,0.000000,9.877467,15.407496,0.0,15.795740,19.136198,0.000000,10.870857,22.254370,...,0.0,11.701971,0.000000,12.784554,11.375386,18.884585,14.448511,0.00000,12.932882,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,TCGA-CD-A489-01A,12.323254,0.000000,16.091254,0.0,16.427109,17.771472,0.000000,11.965715,22.719261,...,0.0,7.018427,0.000000,14.010101,12.475229,17.414922,12.388978,0.00000,13.694892,9.248802
403,TCGA-BR-6566-01A,0.000000,0.000000,16.719802,0.0,16.451107,18.733779,0.000000,10.719868,23.569497,...,0.0,14.880863,0.000000,13.370376,0.000000,19.882176,9.014762,0.00000,14.182239,0.000000
404,TCGA-VQ-A91S-01A,0.000000,6.564124,15.468126,0.0,16.259417,18.217679,0.000000,11.990096,21.839549,...,0.0,13.402292,0.000000,14.370714,10.136451,18.225644,10.857035,0.00000,12.693242,0.000000
405,TCGA-BR-8286-01A,10.967479,7.127704,16.180321,0.0,16.736216,17.407005,10.300027,12.500597,22.774849,...,0.0,11.728282,0.000000,14.823774,11.289263,19.140173,11.524837,0.00000,13.963588,11.230664


In [5]:
# 544명 기증자의 모든 GDC_phenotype 수치
df2

,submitter_id.samples,additional_pharmaceutical_therapy,additional_radiation_therapy,additional_surgery_locoregional_procedure,additional_surgery_metastatic_procedure,age_at_initial_pathologic_diagnosis,anatomic_neoplasm_subdivision_other,antireflux_treatment,barretts_esophagus,batch_number,...,intermediate_dimension.samples,is_ffpe.samples,longest_dimension.samples,oct_embedded.samples,preservation_method.samples,sample_type.samples,sample_type_id.samples,shortest_dimension.samples,state.samples,tissue_type.samples
0,TCGA-CG-5720-01A,NaN,NaN,NaN,NaN,71.0,NaN,NaN,No,95.85.0,...,0.6,False,0.9,NaN,NaN,Primary Tumor,1,0.6,released,Not Reported
1,TCGA-CG-5720-11A,NaN,NaN,NaN,NaN,71.0,NaN,NaN,No,95.85.0,...,0.8,False,1.6,NaN,NaN,Solid Tissue Normal,11,0.4,released,Not Reported
2,TCGA-VQ-A8PX-01A,NaN,NaN,NaN,NaN,51.0,Gastric stump,NO,No,427.44.0,...,NaN,False,NaN,False,NaN,Primary Tumor,1,NaN,released,Not Reported
3,TCGA-BR-A4CR-01A,NaN,NaN,NaN,NaN,70.0,NaN,NaN,NaN,257.69.0,...,NaN,False,NaN,True,NaN,Primary Tumor,1,NaN,released,Not Reported
4,TCGA-MX-A663-01A,NO,NO,NaN,NaN,66.0,Remnant gastrectomy after Billroth,NO,No,321.64.0,...,NaN,False,NaN,True,NaN,Primary Tumor,1,NaN,released,Not Reported
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,TCGA-HU-8243-11A,YES,NaN,NaN,NO,66.0,NaN,NaN,No,220.78.0,...,0.4,False,0.6,NaN,NaN,Solid Tissue Normal,11,0.3,released,Not Reported
540,TCGA-BR-8380-01A,NO,NO,NO,NaN,55.0,NaN,NaN,NaN,220.78.0,...,1.7,False,3.0,NaN,NaN,Primary Tumor,1,0.5,released,Not Reported
541,TCGA-BR-4183-01A,NaN,NaN,NaN,NaN,55.0,NaN,NO,NaN,48.87.0,...,1.4,False,3.0,NaN,NaN,Primary Tumor,1,0.5,released,Not Reported
542,TCGA-BR-4183-11A,NaN,NaN,NaN,NaN,55.0,NaN,NO,NaN,48.87.0,...,1.9,False,2.0,NaN,NaN,Solid Tissue Normal,11,0.4,released,Not Reported


### 0-2. Integrating two files into one using ID as key

In [6]:
# df1 과 df2 데이터의 기증자 수를 맞추려는 노력
df_x = df1.sort_values('Ensembl_ID')
df_x

,Ensembl_ID,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,ENSG00000198242.12,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
354,TCGA-3M-AB46-01A,9.172708,7.329795,17.563194,0.0,16.071659,17.679128,15.409254,13.847298,23.023774,...,0.0,13.609492,0.000000,17.223578,9.908674,18.538086,11.521835,0.0,13.611363,0.0
93,TCGA-3M-AB47-01A,0.000000,7.131909,15.942753,0.0,16.576146,18.984338,0.000000,11.992052,22.195397,...,0.0,16.931121,0.000000,15.015629,11.708390,18.432997,11.943986,0.0,13.446515,0.0
393,TCGA-B7-5818-01A,0.000000,0.000000,16.864073,0.0,17.136882,18.764149,0.000000,8.910704,23.446728,...,0.0,10.022942,0.000000,13.339954,0.000000,19.479232,12.583815,0.0,14.511396,0.0
270,TCGA-B7-A5TI-01A,8.878001,0.000000,16.116272,0.0,17.070254,18.343037,0.000000,9.386831,22.680389,...,0.0,13.837762,7.954758,14.021490,9.613739,18.773215,10.111896,0.0,13.480218,0.0
152,TCGA-B7-A5TJ-01A,0.000000,0.000000,15.842664,0.0,16.094085,18.994900,0.000000,12.888319,22.623489,...,0.0,15.372651,0.000000,14.087535,11.678386,18.732526,11.762020,0.0,13.115669,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,TCGA-VQ-AA6G-01A,0.000000,5.902859,16.295681,0.0,16.489625,18.041383,10.061425,12.960340,21.515821,...,0.0,12.648472,0.000000,12.415254,0.000000,18.653051,12.993535,0.0,12.379185,0.0
276,TCGA-VQ-AA6J-01A,8.274892,6.437561,15.846170,0.0,16.477069,18.576673,0.000000,13.072919,22.461277,...,0.0,12.254013,0.000000,14.911510,10.593089,18.660539,12.851858,0.0,15.844048,0.0
334,TCGA-VQ-AA6K-01A,8.824773,5.994980,16.614839,0.0,16.735731,19.042212,0.000000,11.447325,21.602157,...,0.0,11.534263,7.901634,14.900993,9.560465,19.659519,13.027048,0.0,13.707876,0.0
103,TCGA-ZA-A8F6-01A,8.974378,6.142665,16.158532,0.0,16.715901,18.215005,0.000000,11.724533,21.865307,...,0.0,16.064140,0.000000,15.016143,11.708904,18.120601,12.792334,0.0,13.561190,0.0


In [7]:
# phenotype 데이터 중 악성 종양 등급 데이터만 필요하여 골라 냄
df_y = df2[['submitter_id.samples','neoplasm_histologic_grade']].sort_values('submitter_id.samples')
df_y

,submitter_id.samples,neoplasm_histologic_grade
444,TCGA-3M-AB46-01A,G2
343,TCGA-3M-AB47-01A,G3
242,TCGA-B7-5816-01A,G3
38,TCGA-B7-5818-01A,G3
104,TCGA-B7-A5TI-01A,G3
...,...,...
141,TCGA-VQ-AA6I-01A,G3
519,TCGA-VQ-AA6J-01A,G3
479,TCGA-VQ-AA6K-01A,G3
308,TCGA-ZA-A8F6-01A,G2


In [8]:
# 기증자ID가 일치하는 행만 가지고 데이터프레임을 합침
df_INNER_JOIN = pd.merge(df_y, df_x, left_on='submitter_id.samples', right_on='Ensembl_ID', how='inner')
df_INNER_JOIN

,submitter_id.samples,neoplasm_histologic_grade,Ensembl_ID,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
0,TCGA-3M-AB46-01A,G2,TCGA-3M-AB46-01A,9.172708,7.329795,17.563194,0.0,16.071659,17.679128,15.409254,...,0.0,13.609492,0.000000,17.223578,9.908674,18.538086,11.521835,0.0,13.611363,0.0
1,TCGA-3M-AB47-01A,G3,TCGA-3M-AB47-01A,0.000000,7.131909,15.942753,0.0,16.576146,18.984338,0.000000,...,0.0,16.931121,0.000000,15.015629,11.708390,18.432997,11.943986,0.0,13.446515,0.0
2,TCGA-B7-5818-01A,G3,TCGA-B7-5818-01A,0.000000,0.000000,16.864073,0.0,17.136882,18.764149,0.000000,...,0.0,10.022942,0.000000,13.339954,0.000000,19.479232,12.583815,0.0,14.511396,0.0
3,TCGA-B7-A5TI-01A,G3,TCGA-B7-A5TI-01A,8.878001,0.000000,16.116272,0.0,17.070254,18.343037,0.000000,...,0.0,13.837762,7.954758,14.021490,9.613739,18.773215,10.111896,0.0,13.480218,0.0
4,TCGA-B7-A5TJ-01A,G1,TCGA-B7-A5TJ-01A,0.000000,0.000000,15.842664,0.0,16.094085,18.994900,0.000000,...,0.0,15.372651,0.000000,14.087535,11.678386,18.732526,11.762020,0.0,13.115669,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,TCGA-VQ-AA6G-01A,G2,TCGA-VQ-AA6G-01A,0.000000,5.902859,16.295681,0.0,16.489625,18.041383,10.061425,...,0.0,12.648472,0.000000,12.415254,0.000000,18.653051,12.993535,0.0,12.379185,0.0
403,TCGA-VQ-AA6J-01A,G3,TCGA-VQ-AA6J-01A,8.274892,6.437561,15.846170,0.0,16.477069,18.576673,0.000000,...,0.0,12.254013,0.000000,14.911510,10.593089,18.660539,12.851858,0.0,15.844048,0.0
404,TCGA-VQ-AA6K-01A,G3,TCGA-VQ-AA6K-01A,8.824773,5.994980,16.614839,0.0,16.735731,19.042212,0.000000,...,0.0,11.534263,7.901634,14.900993,9.560465,19.659519,13.027048,0.0,13.707876,0.0
405,TCGA-ZA-A8F6-01A,G2,TCGA-ZA-A8F6-01A,8.974378,6.142665,16.158532,0.0,16.715901,18.215005,0.000000,...,0.0,16.064140,0.000000,15.016143,11.708904,18.120601,12.792334,0.0,13.561190,0.0


In [9]:
# Ensembl_ID 컬럼은 중복되므로 떨구어 줌
df_xy_full = df_INNER_JOIN.drop('Ensembl_ID', axis=1)
df_xy_full

,submitter_id.samples,neoplasm_histologic_grade,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
0,TCGA-3M-AB46-01A,G2,9.172708,7.329795,17.563194,0.0,16.071659,17.679128,15.409254,13.847298,...,0.0,13.609492,0.000000,17.223578,9.908674,18.538086,11.521835,0.0,13.611363,0.0
1,TCGA-3M-AB47-01A,G3,0.000000,7.131909,15.942753,0.0,16.576146,18.984338,0.000000,11.992052,...,0.0,16.931121,0.000000,15.015629,11.708390,18.432997,11.943986,0.0,13.446515,0.0
2,TCGA-B7-5818-01A,G3,0.000000,0.000000,16.864073,0.0,17.136882,18.764149,0.000000,8.910704,...,0.0,10.022942,0.000000,13.339954,0.000000,19.479232,12.583815,0.0,14.511396,0.0
3,TCGA-B7-A5TI-01A,G3,8.878001,0.000000,16.116272,0.0,17.070254,18.343037,0.000000,9.386831,...,0.0,13.837762,7.954758,14.021490,9.613739,18.773215,10.111896,0.0,13.480218,0.0
4,TCGA-B7-A5TJ-01A,G1,0.000000,0.000000,15.842664,0.0,16.094085,18.994900,0.000000,12.888319,...,0.0,15.372651,0.000000,14.087535,11.678386,18.732526,11.762020,0.0,13.115669,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,TCGA-VQ-AA6G-01A,G2,0.000000,5.902859,16.295681,0.0,16.489625,18.041383,10.061425,12.960340,...,0.0,12.648472,0.000000,12.415254,0.000000,18.653051,12.993535,0.0,12.379185,0.0
403,TCGA-VQ-AA6J-01A,G3,8.274892,6.437561,15.846170,0.0,16.477069,18.576673,0.000000,13.072919,...,0.0,12.254013,0.000000,14.911510,10.593089,18.660539,12.851858,0.0,15.844048,0.0
404,TCGA-VQ-AA6K-01A,G3,8.824773,5.994980,16.614839,0.0,16.735731,19.042212,0.000000,11.447325,...,0.0,11.534263,7.901634,14.900993,9.560465,19.659519,13.027048,0.0,13.707876,0.0
405,TCGA-ZA-A8F6-01A,G2,8.974378,6.142665,16.158532,0.0,16.715901,18.215005,0.000000,11.724533,...,0.0,16.064140,0.000000,15.016143,11.708904,18.120601,12.792334,0.0,13.561190,0.0


### 0-3. Making Gene ID list related to STAD

In [10]:
# 위암과 관련 있는 유전자ID 리스트 (지난번888개에 PHGDH(ENSG00000092621)와 단백질 상호작용하는 유전자 359 를 더하고 중복되는 45개와 ID가 없는 17개를 뺀)
df = pd.read_excel('STAD_related_gene_ensenblID_1202.xlsx')
df

,Gene Symbol,Ensembl ID,Gene Symbol_not_found
0,PHGDH,ENSG00000092621,RAD51L3-RFFL
1,APC,ENSG00000134982,LOC126862571
2,MUTYH,ENSG00000132781,LOC126860438
3,IRF1,ENSG00000125347,LOC130001603
4,CASP10,ENSG00000003400,LOC130062899
...,...,...,...
1180,LMBR1L,ENSG00000139636,NaN
1181,CALCOCO2,ENSG00000136436,NaN
1182,HSPE1,ENSG00000115541,NaN
1183,USP7,ENSG00000187555,NaN


### 0-4. Making a dataframe integrating 0-2 and 0-3

In [11]:
df['Ensembl ID'].count()

1185

In [12]:
df_xy_small = df_xy_full[['submitter_id.samples', 'neoplasm_histologic_grade']]
df_xy_small

,submitter_id.samples,neoplasm_histologic_grade
0,TCGA-3M-AB46-01A,G2
1,TCGA-3M-AB47-01A,G3
2,TCGA-B7-5818-01A,G3
3,TCGA-B7-A5TI-01A,G3
4,TCGA-B7-A5TJ-01A,G1
...,...,...
402,TCGA-VQ-AA6G-01A,G2
403,TCGA-VQ-AA6J-01A,G3
404,TCGA-VQ-AA6K-01A,G3
405,TCGA-ZA-A8F6-01A,G2


In [13]:
df['Ensembl ID'].dropna().drop_duplicates()

0       ENSG00000092621
1       ENSG00000134982
2       ENSG00000132781
3       ENSG00000125347
4       ENSG00000003400
             ...       
1180    ENSG00000139636
1181    ENSG00000136436
1182    ENSG00000115541
1183    ENSG00000187555
1184    ENSG00000186868
Name: Ensembl ID, Length: 1185, dtype: object

In [14]:
for E_ID in df['Ensembl ID'].dropna():
    df_xy_small = pd.concat([df_xy_small, df_xy_full.filter(regex=E_ID)], axis=1)
df_xy_small

,submitter_id.samples,neoplasm_histologic_grade,ENSG00000092621.10,ENSG00000134982.15,ENSG00000132781.16,ENSG00000125347.12,ENSG00000003400.13,ENSG00000133703.10,ENSG00000141736.12,ENSG00000066468.19,...,ENSG00000065154.10,ENSG00000140374.14,ENSG00000113649.10,ENSG00000122692.8,ENSG00000146205.12,ENSG00000139636.14,ENSG00000136436.13,ENSG00000115541.9,ENSG00000187555.13,ENSG00000186868.14
0,TCGA-3M-AB46-01A,G2,17.251112,15.079514,17.414938,17.365214,18.054834,17.592039,19.026159,16.988659,...,19.971286,17.520186,17.608391,16.934600,12.232790,15.882418,19.502006,21.471786,19.033745,11.708090
1,TCGA-3M-AB47-01A,G3,14.407931,16.812603,15.904570,18.298847,17.441156,18.036434,17.735773,16.097251,...,18.042885,18.507986,17.605038,17.802902,11.774955,16.386493,18.738287,20.527855,18.332285,10.845279
2,TCGA-B7-5818-01A,G3,13.235416,15.642202,15.912011,18.662111,16.382809,17.797608,19.516644,19.218607,...,18.962656,18.201000,16.951647,17.390941,10.842400,15.912461,18.733346,20.074926,18.111386,9.361220
3,TCGA-B7-A5TI-01A,G3,16.119779,16.060210,16.614935,16.654159,16.730176,17.937752,19.660434,18.737403,...,19.952706,18.530013,17.372864,17.726900,14.901840,15.671850,18.536612,20.684934,18.679611,10.459951
4,TCGA-B7-A5TJ-01A,G1,16.702316,15.719877,16.525847,17.006665,16.876615,17.559441,18.943163,17.838738,...,18.358425,18.338826,17.444754,17.045052,15.390480,16.549372,17.743944,20.391893,18.815926,9.941481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,TCGA-VQ-AA6G-01A,G2,17.216473,17.333125,16.562825,18.292577,19.014720,18.792734,24.963099,15.413158,...,18.727258,18.505930,17.696179,17.312959,15.179511,16.085758,18.788913,20.277917,18.388399,8.246315
403,TCGA-VQ-AA6J-01A,G3,19.585174,16.254265,16.554167,18.734580,17.640446,17.564258,18.318130,16.909481,...,18.491928,17.980797,17.575003,16.999468,14.597397,16.231806,18.553040,20.290658,18.246014,11.735346
404,TCGA-VQ-AA6K-01A,G3,16.326261,16.784325,15.890133,18.029975,17.364764,18.153768,19.371017,16.453277,...,19.215405,18.157770,17.974813,17.032425,13.952300,16.112822,18.251106,19.949184,18.958449,10.287439
405,TCGA-ZA-A8F6-01A,G2,16.193348,16.693416,16.193612,17.997426,17.702167,17.058265,19.367424,16.519393,...,20.106597,17.977469,17.021629,17.271290,12.360294,16.482036,18.290664,19.654904,18.268252,14.064136


In [15]:
df_xy_1000 = df_xy_small.drop(df_xy_small.columns[1002:], axis=1)
df_xy_1000

,submitter_id.samples,neoplasm_histologic_grade,ENSG00000092621.10,ENSG00000134982.15,ENSG00000132781.16,ENSG00000125347.12,ENSG00000003400.13,ENSG00000133703.10,ENSG00000141736.12,ENSG00000066468.19,...,ENSG00000196363.8,ENSG00000183208.11,ENSG00000177889.8,ENSG00000001626.13,ENSG00000168291.11,ENSG00000091140.11,ENSG00000143878.9,ENSG00000028137.15,ENSG00000123416.14,ENSG00000168283.12
0,TCGA-3M-AB46-01A,G2,17.251112,15.079514,17.414938,17.365214,18.054834,17.592039,19.026159,16.988659,...,17.975141,14.629400,18.923630,14.665292,18.303965,18.398376,20.474559,17.548524,19.847513,18.329024
1,TCGA-3M-AB47-01A,G3,14.407931,16.812603,15.904570,18.298847,17.441156,18.036434,17.735773,16.097251,...,18.194745,15.036469,18.385398,18.794606,17.878813,18.641734,21.338569,18.150226,19.944410,18.242066
2,TCGA-B7-5818-01A,G3,13.235416,15.642202,15.912011,18.662111,16.382809,17.797608,19.516644,19.218607,...,18.448118,14.721660,18.822019,19.947795,17.542855,18.407585,19.887796,19.055747,22.132615,17.385839
3,TCGA-B7-A5TI-01A,G3,16.119779,16.060210,16.614935,16.654159,16.730176,17.937752,19.660434,18.737403,...,18.496027,15.547249,18.420841,10.613326,17.221112,18.850662,22.055769,17.145518,21.260584,17.005872
4,TCGA-B7-A5TJ-01A,G1,16.702316,15.719877,16.525847,17.006665,16.876615,17.559441,18.943163,17.838738,...,18.578825,14.342636,17.961348,17.520498,17.469341,18.300016,21.289474,17.769859,20.630573,18.288201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,TCGA-VQ-AA6G-01A,G2,17.216473,17.333125,16.562825,18.292577,19.014720,18.792734,24.963099,15.413158,...,17.388298,15.944085,18.006361,16.725651,17.330812,18.329216,22.221918,18.392274,20.783781,17.575057
403,TCGA-VQ-AA6J-01A,G3,19.585174,16.254265,16.554167,18.734580,17.640446,17.564258,18.318130,16.909481,...,18.622398,14.640067,18.022327,17.282670,17.351236,17.837755,19.020617,19.558533,20.816917,18.769596
404,TCGA-VQ-AA6K-01A,G3,16.326261,16.784325,15.890133,18.029975,17.364764,18.153768,19.371017,16.453277,...,18.580024,14.813767,17.471635,18.157491,17.081141,18.224973,21.848835,17.703319,20.628612,17.322944
405,TCGA-ZA-A8F6-01A,G2,16.193348,16.693416,16.193612,17.997426,17.702167,17.058265,19.367424,16.519393,...,17.698650,15.065811,17.678183,17.995544,17.254952,18.435269,22.391362,18.932874,20.891859,17.736335


In [16]:
df_xy_1000.to_pickle('xy_limit_test_1000.pkl')

In [17]:
df_xy_full.iloc[:,2:1002]

,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,ENSG00000198242.12,ENSG00000259883.1,...,ENSG00000102921.6,ENSG00000167311.12,ENSG00000143178.11,ENSG00000276376.1,ENSG00000163464.7,ENSG00000236246.1,ENSG00000281088.1,ENSG00000254526.1,ENSG00000223575.2,ENSG00000201444.1
0,9.172708,7.329795,17.563194,0.0,16.071659,17.679128,15.409254,13.847298,23.023774,9.054011,...,17.277189,12.166462,15.757780,0.0,10.362207,0.000000,0.0,0.000000,14.546922,0.000000
1,0.000000,7.131909,15.942753,0.0,16.576146,18.984338,0.000000,11.992052,22.195397,10.438084,...,17.560147,11.526851,14.195213,0.0,9.748538,0.000000,0.0,0.000000,12.447442,0.000000
2,0.000000,0.000000,16.864073,0.0,17.136882,18.764149,0.000000,8.910704,23.446728,12.311754,...,17.293539,9.595300,13.415469,0.0,0.000000,0.000000,0.0,10.375666,11.415248,0.000000
3,8.878001,0.000000,16.116272,0.0,17.070254,18.343037,0.000000,9.386831,22.680389,10.756853,...,17.545869,13.295514,14.068784,0.0,10.651695,8.702615,0.0,0.000000,9.446300,0.000000
4,0.000000,0.000000,15.842664,0.0,16.094085,18.994900,0.000000,12.888319,22.623489,11.407562,...,17.601300,9.276286,14.438207,0.0,10.133170,0.000000,0.0,0.000000,11.510775,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,0.000000,5.902859,16.295681,0.0,16.489625,18.041383,10.061425,12.960340,21.515821,12.516175,...,17.981710,9.799512,13.680123,0.0,12.777041,0.000000,0.0,9.259466,9.299541,11.458671
403,8.274892,6.437561,15.846170,0.0,16.477069,18.576673,0.000000,13.072919,22.461277,11.959032,...,17.813007,14.588155,14.114060,0.0,8.465219,0.000000,0.0,0.000000,11.162085,11.001139
404,8.824773,5.994980,16.614839,0.0,16.735731,19.042212,0.000000,11.447325,21.602157,11.510525,...,18.105217,11.739949,14.154335,0.0,11.013190,0.000000,0.0,10.351853,11.198857,0.000000
405,8.974378,6.142665,16.158532,0.0,16.715901,18.215005,0.000000,11.724533,21.865307,11.852985,...,17.699644,11.527365,14.365643,0.0,11.806670,0.000000,0.0,0.000000,11.348715,0.000000


In [19]:
df_xy_2000 = pd.concat([df_xy_1000, df_xy_full.iloc[:,2:1002]], axis=1)

In [20]:
df_xy_2000.to_pickle('xy_limit_test_2000.pkl')

In [21]:
df_xy_3000 = pd.concat([df_xy_1000, df_xy_full.iloc[:,2:2002]], axis=1)
df_xy_3000.to_pickle('xy_limit_test_3000.pkl')

In [107]:
df_xy_4000 = pd.concat([df_xy_1000, df_xy_full.iloc[:,2:3002]], axis=1)
df_xy_4000.to_pickle('xy_limit_test_4000.pkl')

In [134]:
df_xy_8000 = pd.concat([df_xy_1000, df_xy_full.iloc[:,2:7002]], axis=1)
df_xy_8000.to_pickle('xy_limit_test_8000.pkl')

In [161]:
df_xy_16000 = pd.concat([df_xy_1000, df_xy_full.iloc[:,2:15002]], axis=1)
df_xy_16000.to_pickle('xy_limit_test_16000.pkl')

In [188]:
df_xy_20000 = pd.concat([df_xy_1000, df_xy_full.iloc[:,2:19002]], axis=1)
df_xy_20000.to_pickle('xy_limit_test_20000.pkl')

In [215]:
df_xy_19000 = pd.concat([df_xy_1000, df_xy_full.iloc[:,2:18002]], axis=1)
df_xy_19000.to_pickle('xy_limit_test_19000.pkl')

In [243]:
df_xy_18000 = pd.concat([df_xy_1000, df_xy_full.iloc[:,2:17002]], axis=1)
df_xy_18000.to_pickle('xy_limit_test_18000.pkl')

## 1. Making the final numpy dataset of XY values for inserting into model

In [2]:
import pandas as pd
import numpy as np
import h5py

### 1-1. Trim & Sort

In [244]:
df = pd.read_pickle('xy_limit_test_18000.pkl')
df

,submitter_id.samples,neoplasm_histologic_grade,ENSG00000092621.10,ENSG00000134982.15,ENSG00000132781.16,ENSG00000125347.12,ENSG00000003400.13,ENSG00000133703.10,ENSG00000141736.12,ENSG00000066468.19,...,ENSG00000081237.17,ENSG00000143315.6,ENSG00000271893.1,ENSG00000163507.12,ENSG00000225564.5,ENSG00000236853.1,ENSG00000110680.11,ENSG00000256120.1,ENSG00000261240.1,ENSG00000274230.1
0,TCGA-3M-AB46-01A,G2,17.251112,15.079514,17.414938,17.365214,18.054834,17.592039,19.026159,16.988659,...,14.939418,18.374368,0.000000,16.389474,6.547234,0.000000,0.000000,0.000000,11.394490,0.0
1,TCGA-3M-AB47-01A,G3,14.407931,16.812603,15.904570,18.298847,17.441156,18.036434,17.735773,16.097251,...,17.453493,16.560958,9.470289,16.082772,0.000000,0.000000,7.395977,10.556913,9.611623,0.0
2,TCGA-B7-5818-01A,G3,13.235416,15.642202,15.912011,18.662111,16.382809,17.797608,19.516644,19.218607,...,16.853878,16.240631,0.000000,16.035882,0.000000,0.000000,0.000000,0.000000,10.485211,0.0
3,TCGA-B7-A5TI-01A,G3,16.119779,16.060210,16.614935,16.654159,16.730176,17.937752,19.660434,18.737403,...,16.508963,16.353621,0.000000,16.223355,0.000000,0.000000,0.000000,0.000000,11.514209,0.0
4,TCGA-B7-A5TJ-01A,G1,16.702316,15.719877,16.525847,17.006665,16.876615,17.559441,18.943163,17.838738,...,14.624703,17.016545,10.439281,15.650611,0.000000,8.580379,0.000000,0.000000,12.749900,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,TCGA-VQ-AA6G-01A,G2,17.216473,17.333125,16.562825,18.292577,19.014720,18.792734,24.963099,15.413158,...,15.514985,16.284293,10.226415,15.828367,0.000000,0.000000,7.154505,0.000000,12.536903,0.0
403,TCGA-VQ-AA6J-01A,G3,19.585174,16.254265,16.554167,18.734580,17.640446,17.564258,18.318130,16.909481,...,18.639702,16.613072,10.768315,16.479702,0.000000,11.076445,0.000000,9.856746,0.000000,0.0
404,TCGA-VQ-AA6K-01A,G3,16.326261,16.784325,15.890133,18.029975,17.364764,18.153768,19.371017,16.453277,...,16.721011,16.860650,0.000000,15.815466,9.185586,10.043543,11.486411,10.407613,12.460629,0.0
405,TCGA-ZA-A8F6-01A,G2,16.193348,16.693416,16.193612,17.997426,17.702167,17.058265,19.367424,16.519393,...,17.337190,16.770791,0.000000,14.863614,0.000000,10.929800,0.000000,8.974378,11.932589,0.0


In [245]:
df.columns[2:]

Index(['ENSG00000092621.10', 'ENSG00000134982.15', 'ENSG00000132781.16',
       'ENSG00000125347.12', 'ENSG00000003400.13', 'ENSG00000133703.10',
       'ENSG00000141736.12', 'ENSG00000066468.19', 'ENSG00000121879.3',
       'ENSG00000067082.13',
       ...
       'ENSG00000081237.17', 'ENSG00000143315.6', 'ENSG00000271893.1',
       'ENSG00000163507.12', 'ENSG00000225564.5', 'ENSG00000236853.1',
       'ENSG00000110680.11', 'ENSG00000256120.1', 'ENSG00000261240.1',
       'ENSG00000274230.1'],
      dtype='object', length=18000)

In [246]:
# 유전자아이디에서 점이하를 버린 리스트를 만들고
gene_ids_trimmed = [gene_id[:15] for gene_id in df.columns[2:]]

In [247]:
# 그 리스트를 가지고 데이터프레임을 다시 만듦
for original, trimmed in zip(df.columns[2:], gene_ids_trimmed):
    df.rename(columns={original:trimmed}, inplace=True)
df

,submitter_id.samples,neoplasm_histologic_grade,ENSG00000092621,ENSG00000134982,ENSG00000132781,ENSG00000125347,ENSG00000003400,ENSG00000133703,ENSG00000141736,ENSG00000066468,...,ENSG00000081237,ENSG00000143315,ENSG00000271893,ENSG00000163507,ENSG00000225564,ENSG00000236853,ENSG00000110680,ENSG00000256120,ENSG00000261240,ENSG00000274230
0,TCGA-3M-AB46-01A,G2,17.251112,15.079514,17.414938,17.365214,18.054834,17.592039,19.026159,16.988659,...,14.939418,18.374368,0.000000,16.389474,6.547234,0.000000,0.000000,0.000000,11.394490,0.0
1,TCGA-3M-AB47-01A,G3,14.407931,16.812603,15.904570,18.298847,17.441156,18.036434,17.735773,16.097251,...,17.453493,16.560958,9.470289,16.082772,0.000000,0.000000,7.395977,10.556913,9.611623,0.0
2,TCGA-B7-5818-01A,G3,13.235416,15.642202,15.912011,18.662111,16.382809,17.797608,19.516644,19.218607,...,16.853878,16.240631,0.000000,16.035882,0.000000,0.000000,0.000000,0.000000,10.485211,0.0
3,TCGA-B7-A5TI-01A,G3,16.119779,16.060210,16.614935,16.654159,16.730176,17.937752,19.660434,18.737403,...,16.508963,16.353621,0.000000,16.223355,0.000000,0.000000,0.000000,0.000000,11.514209,0.0
4,TCGA-B7-A5TJ-01A,G1,16.702316,15.719877,16.525847,17.006665,16.876615,17.559441,18.943163,17.838738,...,14.624703,17.016545,10.439281,15.650611,0.000000,8.580379,0.000000,0.000000,12.749900,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,TCGA-VQ-AA6G-01A,G2,17.216473,17.333125,16.562825,18.292577,19.014720,18.792734,24.963099,15.413158,...,15.514985,16.284293,10.226415,15.828367,0.000000,0.000000,7.154505,0.000000,12.536903,0.0
403,TCGA-VQ-AA6J-01A,G3,19.585174,16.254265,16.554167,18.734580,17.640446,17.564258,18.318130,16.909481,...,18.639702,16.613072,10.768315,16.479702,0.000000,11.076445,0.000000,9.856746,0.000000,0.0
404,TCGA-VQ-AA6K-01A,G3,16.326261,16.784325,15.890133,18.029975,17.364764,18.153768,19.371017,16.453277,...,16.721011,16.860650,0.000000,15.815466,9.185586,10.043543,11.486411,10.407613,12.460629,0.0
405,TCGA-ZA-A8F6-01A,G2,16.193348,16.693416,16.193612,17.997426,17.702167,17.058265,19.367424,16.519393,...,17.337190,16.770791,0.000000,14.863614,0.000000,10.929800,0.000000,8.974378,11.932589,0.0


In [248]:
# 컬럼 이름 오름차순으로 데이터프레임 정렬(나중에 새로운 데이터로 추론시에 유전자아이디가 섞여 있으면 안 되기 때문)
dfs = df.sort_index(axis=1)
dfs

,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001460,ENSG00000001626,ENSG00000001629,ENSG00000001630,ENSG00000002330,ENSG00000002549,ENSG00000002745,...,ENSGR0000227159,ENSGR0000228410,ENSGR0000234958,ENSGR0000236017,ENSGR0000237801,ENSGR0000264510,ENSGR0000265658,ENSGR0000277120,neoplasm_histologic_grade,submitter_id.samples
0,15.369606,20.220979,17.451362,14.197870,14.665292,17.783249,15.192722,18.727956,17.938825,9.557661,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,G2,TCGA-3M-AB46-01A
1,18.746885,18.500387,17.074325,14.691390,18.794606,17.383192,13.816860,18.090443,19.228355,10.579867,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,G3,TCGA-3M-AB47-01A
2,15.728434,19.761280,17.298747,14.667178,19.947795,17.286428,16.759505,19.709574,19.935341,7.653180,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,G3,TCGA-B7-5818-01A
3,17.205885,19.181518,17.073214,15.588439,10.613326,17.778447,15.639806,18.942790,19.230782,9.998846,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,G3,TCGA-B7-A5TI-01A
4,15.468113,19.586652,17.343706,14.781760,17.520498,17.259537,16.910285,18.527097,18.481142,9.743241,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,G1,TCGA-B7-A5TJ-01A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,16.164322,18.996392,18.266418,15.886003,16.725651,18.349770,16.872743,17.717748,19.549115,9.338112,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,G2,TCGA-VQ-AA6G-01A
403,16.884962,18.794649,18.807228,15.414743,17.282670,18.390982,15.048922,17.603730,19.958540,11.115848,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,G3,TCGA-VQ-AA6J-01A
404,18.834288,19.295330,17.097678,14.390502,18.157491,17.708813,16.250878,17.706194,19.835024,10.623079,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,G3,TCGA-VQ-AA6K-01A
405,19.260751,18.647983,17.111343,14.149700,17.995544,17.593330,15.635856,18.393344,18.931060,11.165029,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,G2,TCGA-ZA-A8F6-01A


In [249]:
# 정답값이 되는 악성 종양 등급 별로 행 정렬(훈련,검증데이터로 나눌 때 필요)
dfs2 = dfs.sort_values(by='neoplasm_histologic_grade')
dfs2

,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001460,ENSG00000001626,ENSG00000001629,ENSG00000001630,ENSG00000002330,ENSG00000002549,ENSG00000002745,...,ENSGR0000227159,ENSGR0000228410,ENSGR0000234958,ENSGR0000236017,ENSGR0000237801,ENSGR0000264510,ENSGR0000265658,ENSGR0000277120,neoplasm_histologic_grade,submitter_id.samples
318,15.378928,20.219629,17.741417,14.171273,16.825263,17.418912,14.476502,19.121479,18.869140,11.285698,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,G1,TCGA-KB-A93H-01A
317,18.931779,18.819095,16.358518,13.730371,12.144605,17.949975,14.746452,17.963353,18.735052,10.385346,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,G1,TCGA-KB-A93G-01A
132,14.975890,19.807727,16.668176,15.105117,17.048517,17.140430,14.665844,19.515146,19.674336,10.441765,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,G1,TCGA-CD-5800-01A
179,17.513226,18.941061,16.626200,14.824292,15.740072,16.887472,15.853202,19.171449,19.253198,9.646290,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,G1,TCGA-CG-5719-01A
258,13.710204,19.919778,16.700326,14.791969,18.664863,17.310924,16.134362,18.456981,19.019208,10.173965,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,G1,TCGA-HU-8244-01A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,17.722751,18.749838,16.836165,14.909098,14.442284,17.683470,15.826311,18.851856,18.823866,10.470715,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GX,TCGA-BR-8297-01A
378,15.967653,20.160689,17.730010,13.771971,19.391947,18.449575,15.052617,17.841738,20.033906,13.388739,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GX,TCGA-VQ-A91S-01A
344,16.746239,19.631296,16.479393,14.211426,11.517487,17.778711,15.216168,18.765269,20.625188,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GX,TCGA-SW-A7EA-01A
50,15.518629,19.331497,17.446486,14.067786,17.675182,17.700147,16.262780,18.370031,18.131626,10.243454,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GX,TCGA-BR-7197-01A


### 1-2. Extracting dataset X & Y

In [250]:
df_X = dfs2.drop(labels=['neoplasm_histologic_grade','submitter_id.samples'], axis=1)
df_X

,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001460,ENSG00000001626,ENSG00000001629,ENSG00000001630,ENSG00000002330,ENSG00000002549,ENSG00000002745,...,ENSGR0000223773,ENSGR0000226179,ENSGR0000227159,ENSGR0000228410,ENSGR0000234958,ENSGR0000236017,ENSGR0000237801,ENSGR0000264510,ENSGR0000265658,ENSGR0000277120
318,15.378928,20.219629,17.741417,14.171273,16.825263,17.418912,14.476502,19.121479,18.869140,11.285698,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
317,18.931779,18.819095,16.358518,13.730371,12.144605,17.949975,14.746452,17.963353,18.735052,10.385346,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
132,14.975890,19.807727,16.668176,15.105117,17.048517,17.140430,14.665844,19.515146,19.674336,10.441765,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
179,17.513226,18.941061,16.626200,14.824292,15.740072,16.887472,15.853202,19.171449,19.253198,9.646290,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
258,13.710204,19.919778,16.700326,14.791969,18.664863,17.310924,16.134362,18.456981,19.019208,10.173965,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,17.722751,18.749838,16.836165,14.909098,14.442284,17.683470,15.826311,18.851856,18.823866,10.470715,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
378,15.967653,20.160689,17.730010,13.771971,19.391947,18.449575,15.052617,17.841738,20.033906,13.388739,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
344,16.746239,19.631296,16.479393,14.211426,11.517487,17.778711,15.216168,18.765269,20.625188,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50,15.518629,19.331497,17.446486,14.067786,17.675182,17.700147,16.262780,18.370031,18.131626,10.243454,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [251]:
df_X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 407 entries, 318 to 66
Columns: 18000 entries, ENSG00000000971 to ENSGR0000277120
dtypes: float64(18000)
memory usage: 55.9 MB


In [252]:
df_Y = dfs2[['neoplasm_histologic_grade']]
df_Y

,neoplasm_histologic_grade
318,G1
317,G1
132,G1
179,G1
258,G1
...,...
82,GX
378,GX
344,GX
50,GX


In [253]:
df_Y.info()

<class 'pandas.core.frame.DataFrame'>
Index: 407 entries, 318 to 66
Data columns (total 1 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   neoplasm_histologic_grade  407 non-null    object
dtypes: object(1)
memory usage: 6.4+ KB


In [254]:
df_Y.shape

(407, 1)

In [255]:
print('G1:  ', len(df_Y[df_Y['neoplasm_histologic_grade']=='G1']))
print('G2: ', len(df_Y[df_Y['neoplasm_histologic_grade']=='G2']))
print('G3: ', len(df_Y[df_Y['neoplasm_histologic_grade']=='G3']))
print('GX:   ', len(df_Y[df_Y['neoplasm_histologic_grade']=='GX']))

G1:   10
G2:  150
G3:  238
GX:    9


In [256]:
# NLL Loss 계산 시 문제가 생겨 label 차원은 2차원 행렬이 아닌 평범한 1차원 벡터가 되게 바꿈 
df_Y = dfs2['neoplasm_histologic_grade']
df_Y

318    G1
317    G1
132    G1
179    G1
258    G1
       ..
82     GX
378    GX
344    GX
50     GX
66     GX
Name: neoplasm_histologic_grade, Length: 407, dtype: object

### 1-3. Splitting dataset into train, val, test 

In [257]:
# X, Y 데이터를 약 7 : 2 : 1 의 비율로 나눔
df_X_train = pd.concat([df_X[0: 7], df_X[ 10:115], df_X[160:327], df_X[398:404]])
df_X_val   = pd.concat([df_X[7: 9], df_X[115:145], df_X[327:375], df_X[404:406]])
df_X_test  = pd.concat([df_X[9:10], df_X[145:160], df_X[375:398], df_X[406:407]])
df_Y_train = pd.concat([df_Y[0: 7], df_Y[ 10:115], df_Y[160:327], df_Y[398:404]])
df_Y_val   = pd.concat([df_Y[7: 9], df_Y[115:145], df_Y[327:375], df_Y[404:406]])
df_Y_test  = pd.concat([df_Y[9:10], df_Y[145:160], df_Y[375:398], df_Y[406:407]])

In [258]:
len(df_Y_train)+len(df_Y_val)+len(df_Y_test)

407

In [259]:
# X 데이터 : pandas dataframe 형식에서 numpy array 형식으로 변환
X_train = df_X_train.to_numpy(dtype='float64')
X_val = df_X_val.to_numpy(dtype='float64')
X_test = df_X_test.to_numpy(dtype='float64')
X_train.shape, X_train[0].dtype

((285, 18000), dtype('float64'))

In [260]:
# Y 데이터 : 문자에서 정수로 변환 G1->1, G2->2, G3->3, GX->0
Y_train = df_Y_train.replace({'G1':1, 'G2':2, 'G3':3, 'GX':0}).to_numpy(dtype='int8')
Y_val = df_Y_val.replace({'G1':1, 'G2':2, 'G3':3, 'GX':0}).to_numpy(dtype='int8')
Y_test = df_Y_test.replace({'G1':1, 'G2':2, 'G3':3, 'GX':0}).to_numpy(dtype='int8')
Y_train.shape, Y_train[0].dtype

((285,), dtype('int8'))

### 1-4. Min-Max Normalization & Expansion for richer transformer embedding

In [261]:
np.min(X_train), np.max(X_train)

(0.0, 31.18721908083413)

In [262]:
def MinMax(X):
    X_norm = (X-np.min(X)) / (np.max(X)-np.min(X))
    return X_norm

In [263]:
np.min(MinMax(X_train)), np.max(MinMax(X_train))

(0.0, 1.0)

In [264]:
X_train_norm = MinMax(X_train) * 319
X_train_norm

array([[157.30412073, 206.81746899, 181.46895853, ...,   0.        ,
          0.        ,   0.        ],
       [193.64463369, 192.49203866, 167.32390503, ...,   0.        ,
          0.        ,   0.        ],
       [153.18162344, 202.60430677, 170.49125195, ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [190.80704215, 188.07301827, 172.01598686, ...,   0.        ,
          0.        ,   0.        ],
       [181.27803199, 191.78363413, 172.2095379 , ...,   0.        ,
          0.        ,   0.        ],
       [163.32591956, 206.21459739, 181.35228079, ...,   0.        ,
          0.        ,   0.        ]])

In [265]:
np.min(X_train_norm), np.max(X_train_norm)

(0.0, 319.0)

In [266]:
X_train_norm = MinMax(X_train) * 319
X_val_norm = MinMax(X_val) * 319
X_test_norm = MinMax(X_test) * 319

### 1-5. Save datasets in 2 different ways

In [267]:
X_Train = np.concatenate([X_train_norm, X_test_norm])
X_Train.shape

(325, 18000)

In [268]:
Y_Train = np.concatenate([Y_train, Y_test])
Y_Train.shape

(325,)

In [269]:
# 2. 위 train+test 를 합친 전체의 80% 인 Train 데이터파일 
hf = h5py.File('STAD_Dataset2_Train_feature18000.h5', 'w')
hf.create_dataset('X_Train', data=X_Train)
hf.create_dataset('Y_Train', data=Y_Train)
hf.close()

In [106]:
# 3. 나머지 20% 만 가진 Test 데이터파일 
hf = h5py.File('STAD_Dataset2_Test.h5', 'w')
hf.create_dataset('X_Test', data=X_val_norm)
hf.create_dataset('Y_Test', data=Y_val)
hf.close()

### 1-6. Dataset Validation Check

In [242]:
hf = h5py.File('STAD_Dataset2_Train_feature19000.h5', 'r')

X_Train = np.array(hf.get('X_Train'))
Y_Train = np.array(hf.get('Y_Train'))
print(hf.keys())
print('X_Train.shape : ', X_Train.shape)
print('Y_Train.shape : ', Y_Train.shape)
print('X_Train' in hf.keys())

hf.close()

X_Train.dtype, Y_Train.dtype

<KeysViewHDF5 ['X_Train', 'Y_Train']>
X_Train.shape :  (325, 19000)
Y_Train.shape :  (325,)
True


(dtype('<f8'), dtype('int8'))

In [52]:
hf = h5py.File('STAD_Dataset2_Test_feature1000.h5', 'r')
# hf.keys()
X_Test = np.array(hf.get('X_Test'))
Y_Test = np.array(hf.get('Y_Test'))

print('X_Test.shape  : ', X_Test.shape)
print('Y_Test.shape  : ', Y_Test.shape)

hf.close()

X_Test.dtype, Y_Test.dtype

X_Test.shape  :  (82, 1000)
Y_Test.shape  :  (82,)


(dtype('<f8'), dtype('int8'))